In [ ]:
## TAAS 교통사고데이터 좌표 크롤러 with Selenium
#Selenium을 이용한 동적 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from urllib import parse
import time
import pandas as pd
import re
from tqdm import tqdm

C:\Users\sons\AppData\Local\Temp\ipykernel_41832\3160177533.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [28]:
# TAAS GIS 시스템 로드
driver = webdriver.Chrome()
driver.maximize_window()
driver.get('http://taas.koroad.or.kr/gis/mcm/mcl/initMap.do?menuId=GIS_GMP_STS_RSN')
time.sleep(1)

In [33]:
#사고분석 크롤링 시도(용인시 고속도로)

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 체크박스 체크
elements = driver.find_elements(By.NAME, 'ACDNT_GAE_CODE')[0:4]
for element in elements:
    if not element.is_selected():
        element.click()

selectbox = driver.find_element(By.ID, 'ptsRafSido')
select = Select(selectbox)
select.select_by_visible_text("경기도")

selectbox = driver.find_element(By.ID, 'ptsRafSigungu')

select = Select(selectbox)
select.select_by_visible_text("용인시")

# 도로종류 버튼 클릭
driver.find_element(By.XPATH, '//*[@id="ptsRaf-ROAD_TYPE"]/a').send_keys(Keys.ENTER)
WebDriverWait(driver, 10)

# 전체해제 클릭
checkbox_deselect = driver.find_element(By.ID, 'chk_deselect')
checkbox_deselect.click()

# 고속국도 체크
checkbox_highway = driver.find_element(By.XPATH, "//input[@name='ROAD_TYPE' and @value='6']")
checkbox_highway.click()
WebDriverWait(driver, 5)

# 닫기 버튼 클릭
button_close = driver.find_element(By.ID, 'chk_ok')
button_close.click()

time.sleep(1)




ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=120.0.6099.225)
Stacktrace:
	GetHandleVerifier [0x00007FF703E62142+3514994]
	(No symbol) [0x00007FF703A80CE2]
	(No symbol) [0x00007FF7039274C3]
	(No symbol) [0x00007FF703972D29]
	(No symbol) [0x00007FF703966A0F]
	(No symbol) [0x00007FF703995FEA]
	(No symbol) [0x00007FF7039663B6]
	(No symbol) [0x00007FF703996490]
	(No symbol) [0x00007FF7039B28F6]
	(No symbol) [0x00007FF703995D93]
	(No symbol) [0x00007FF703964BDC]
	(No symbol) [0x00007FF703965C64]
	GetHandleVerifier [0x00007FF703E8E16B+3695259]
	GetHandleVerifier [0x00007FF703EE6737+4057191]
	GetHandleVerifier [0x00007FF703EDE4E3+4023827]
	GetHandleVerifier [0x00007FF703BB04F9+689705]
	(No symbol) [0x00007FF703A8C048]
	(No symbol) [0x00007FF703A88044]
	(No symbol) [0x00007FF703A881C9]
	(No symbol) [0x00007FF703A788C4]
	BaseThreadInitThunk [0x00007FFDF0337614+20]
	RtlUserThreadStart [0x00007FFDF1AC26A1+33]


In [35]:
# search
driver.find_element(By.XPATH, '//*[@id="regionAccidentFind"]/div/p/a').send_keys(Keys.ENTER)
time.sleep(1)

In [36]:
# Crawling
coord_ls = []
srs_ls = []

for i in tqdm(range(0,33698)):
    driver.execute_script(f'gis.srh.msh.selectListUi({i}, 11);') # 개별 사고지점
    time.sleep(1) # 로드

    bbox = driver.find_elements(By.CLASS_NAME,'olTileImage')[-1]
    # 디버깅 코드
    # if isinstance(bbox, list):
    #     print("Debugging - src content:")
    #     for element in bbox:
    #         print(element)
    # else:
    #     print("Debugging - src content:", bbox)

    src = parse.unquote(bbox.get_attribute('src')) # get src and unquote from 16byte

    # 디버깅 코드
    # if isinstance(src, list):
    #     print("Debugging - src content:")
    #     for element in src:
    #         print(element)
    # else:
    #     print("Debugging - src content:", src)

    src_split = src.split('&')
    print("src_split : " + str(src_split))  # Use str() to convert list to string

    # if isinstance(src_split, list):
    #     print("Debugging - src_split:")
    #     for element in src_split:
    #         print(element)
    # else:
    #     print("Debugging - src_split:", src_split)


    bbox_text = src_split[-3][5:]
    print("bbox_text : " + bbox_text)


    coords = [float(i) for i in bbox_text.split(',')]
    x, y = (coords[0]+coords[2])/2, (coords[1]+coords[3])/2 # center position
    srs = src_split[-4][4:]
    coord_ls.append([x, y])
    srs_ls.append(srs)

  0%|          | 1/33698 [00:01<9:49:49,  1.05s/it]

Debugging - src content: <selenium.webdriver.remote.webelement.WebElement (session="5b06b3d12ca3a76e5c613afa896de233", element="9E340076E16B0B12E9EB60FAF1C429D4_element_4939")>
Debugging - src content: https://taas.koroad.or.kr/gis36/TAAS/wms?LAYERS=GNRL_GIS_ACDNT_INFO_NON&STYLES=&FORMAT=image/png&TRANSPARENT=TRUE&TILED=false&TILESORIGIN=-30000,-60000&PROJECTION=EPSG:5179&DISPLAYPROJECTION=EPSG:5181&TRANSFORM=EPSG:5181&SERVICE=WMS&VERSION=1.1.1&REQUEST=GetMap&CQL_FILTER=(engn_code='00') and (acdnt_year between '2022' and '2022')  and (legaldong_code ilike '4146%') and (( acdnt_gae_code = '01') or ( acdnt_gae_code = '02') or ( acdnt_gae_code = '03') or ( acdnt_gae_code = '04')) and (( road_type = '6'))&SRS=EPSG:5181&BBOX=158040.08898455,388474.69582018,260312.08898455,448314.69582018&WIDTH=1598&HEIGHT=935
src_split : ['https://taas.koroad.or.kr/gis36/TAAS/wms?LAYERS=GNRL_GIS_ACDNT_INFO_NON', 'STYLES=', 'FORMAT=image/png', 'TRANSPARENT=TRUE', 'TILED=false', 'TILESORIGIN=-30000,-60000', '

  0%|          | 2/33698 [00:02<9:45:21,  1.04s/it]

Debugging - src content: <selenium.webdriver.remote.webelement.WebElement (session="5b06b3d12ca3a76e5c613afa896de233", element="9E340076E16B0B12E9EB60FAF1C429D4_element_5389")>
Debugging - src content: https://taas.koroad.or.kr/gis36/TAAS/wms?LAYERS=GNRL_GIS_ACDNT_INFO_NON&STYLES=&FORMAT=image/png&TRANSPARENT=TRUE&TILED=false&TILESORIGIN=-30000,-60000&PROJECTION=EPSG:5179&DISPLAYPROJECTION=EPSG:5181&TRANSFORM=EPSG:5181&SERVICE=WMS&VERSION=1.1.1&REQUEST=GetMap&CQL_FILTER=(engn_code='00') and (acdnt_year between '2022' and '2022')  and (legaldong_code ilike '4146%') and (( acdnt_gae_code = '01') or ( acdnt_gae_code = '02') or ( acdnt_gae_code = '03') or ( acdnt_gae_code = '04')) and (( road_type = '6'))&SRS=EPSG:5181&BBOX=158041.22091531,396583.94392474,260313.22091531,456423.94392474&WIDTH=1598&HEIGHT=935
src_split : ['https://taas.koroad.or.kr/gis36/TAAS/wms?LAYERS=GNRL_GIS_ACDNT_INFO_NON', 'STYLES=', 'FORMAT=image/png', 'TRANSPARENT=TRUE', 'TILED=false', 'TILESORIGIN=-30000,-60000', '

  0%|          | 3/33698 [00:03<9:43:55,  1.04s/it]

...

  1%|          | 267/33698 [04:36<9:35:39,  1.03s/it]

Debugging - src content: <selenium.webdriver.remote.webelement.WebElement (session="5b06b3d12ca3a76e5c613afa896de233", element="9E340076E16B0B12E9EB60FAF1C429D4_element_124301")>
Debugging - src content: https://taas.koroad.or.kr/gis36/TAAS/wms?LAYERS=GNRL_GIS_ACDNT_INFO_NON&STYLES=&FORMAT=image/png&TRANSPARENT=TRUE&TILED=false&TILESORIGIN=-30000,-60000&PROJECTION=EPSG:5179&DISPLAYPROJECTION=EPSG:5181&TRANSFORM=EPSG:5181&SERVICE=WMS&VERSION=1.1.1&REQUEST=GetMap&CQL_FILTER=(engn_code='00') and (acdnt_year between '2022' and '2022')  and (legaldong_code ilike '4146%') and (( acdnt_gae_code = '01') or ( acdnt_gae_code = '02') or ( acdnt_gae_code = '03') or ( acdnt_gae_code = '04')) and (( road_type = '6'))&SRS=EPSG:5181&BBOX=154056.56415219,395463.41866339,256328.56415219,455303.41866339&WIDTH=1598&HEIGHT=935
src_split : ['https://taas.koroad.or.kr/gis36/TAAS/wms?LAYERS=GNRL_GIS_ACDNT_INFO_NON', 'STYLES=', 'FORMAT=image/png', 'TRANSPARENT=TRUE', 'TILED=false', 'TILESORIGIN=-30000,-60000',

  1%|          | 268/33698 [04:37<9:36:54,  1.04s/it]

Debugging - src content: <selenium.webdriver.remote.webelement.WebElement (session="5b06b3d12ca3a76e5c613afa896de233", element="9E340076E16B0B12E9EB60FAF1C429D4_element_124751")>
Debugging - src content: https://taas.koroad.or.kr/gis36/TAAS/wms?LAYERS=GNRL_GIS_ACDNT_INFO_NON&STYLES=&FORMAT=image/png&TRANSPARENT=TRUE&TILED=false&TILESORIGIN=-30000,-60000&PROJECTION=EPSG:5179&DISPLAYPROJECTION=EPSG:5181&TRANSFORM=EPSG:5181&SERVICE=WMS&VERSION=1.1.1&REQUEST=GetMap&CQL_FILTER=(engn_code='00') and (acdnt_year between '2022' and '2022')  and (legaldong_code ilike '4146%') and (( acdnt_gae_code = '01') or ( acdnt_gae_code = '02') or ( acdnt_gae_code = '03') or ( acdnt_gae_code = '04')) and (( road_type = '6'))&SRS=EPSG:5181&BBOX=157976.72195724,388539.38697539,260248.72195724,448379.38697539&WIDTH=1598&HEIGHT=935
src_split : ['https://taas.koroad.or.kr/gis36/TAAS/wms?LAYERS=GNRL_GIS_ACDNT_INFO_NON', 'STYLES=', 'FORMAT=image/png', 'TRANSPARENT=TRUE', 'TILED=false', 'TILESORIGIN=-30000,-60000',

JavascriptException: Message: javascript error: Cannot read properties of undefined (reading 'layer')
  (Session info: chrome=120.0.6099.225)
Stacktrace:
	GetHandleVerifier [0x00007FF703E62142+3514994]
	(No symbol) [0x00007FF703A80CE2]
	(No symbol) [0x00007FF7039276AA]
	(No symbol) [0x00007FF70392CFC0]
	(No symbol) [0x00007FF70392F6F7]
	(No symbol) [0x00007FF7039B316B]
	(No symbol) [0x00007FF703995FEA]
	(No symbol) [0x00007FF7039B28F6]
	(No symbol) [0x00007FF703995D93]
	(No symbol) [0x00007FF703964BDC]
	(No symbol) [0x00007FF703965C64]
	GetHandleVerifier [0x00007FF703E8E16B+3695259]
	GetHandleVerifier [0x00007FF703EE6737+4057191]
	GetHandleVerifier [0x00007FF703EDE4E3+4023827]
	GetHandleVerifier [0x00007FF703BB04F9+689705]
	(No symbol) [0x00007FF703A8C048]
	(No symbol) [0x00007FF703A88044]
	(No symbol) [0x00007FF703A881C9]
	(No symbol) [0x00007FF703A788C4]
	BaseThreadInitThunk [0x00007FFDF0337614+20]
	RtlUserThreadStart [0x00007FFDF1AC26A1+33]


In [37]:
# 크롤링한 좌표 확인
coord_ls

[[209176.08898455, 418394.69582018],
 [209177.22091531, 426503.94392474],
 [224865.49670994002, 415932.53885942],
 [209216.39344077, 420989.94633964],
 [209160.0626828, 421615.89888425],
 [209211.80591178, 420154.58809908],
 [209034.19202678, 418743.0855474],
 [210759.24722064, 399705.55622113],
 [211087.79599158, 421244.93875139],
 [209148.37076302, 413982.78593158],
 [209296.5676287, 415851.31513254],
 [209035.03380589, 418773.1019889],
 [205165.5649273, 425192.20022866],
 [209183.49846377, 423048.59556201],
 [209193.60893736, 419244.12795092],
 [209079.03127552, 413852.36790157],
 [209170.01590082, 420868.65275342],
 [209167.8643837, 424112.93843643],
 [209283.6895888, 415827.23751964],
 [209437.32518879, 414960.70195528],
 [208964.41040531, 418885.77394688],
 [209125.07659399, 418582.5014427],
 [217168.36000277998, 418386.90708926],
 [209688.08860879, 421272.55228576],
 [222317.57082337, 416286.23450135],
 [209099.45937201, 418697.41205622],
 [209339.70499653, 414312.92765014],
 [2

In [38]:
# 크롤링데이터 json 파일로 저장
import json

# JSON 파일로 저장
with open('yongin_list.json', 'w') as json_file:
    json.dump(coord_ls, json_file)


In [39]:
# 크롤링한 좌표 데이터 갯수 체크
import json

# 파일 이름
file_name = 'yongin_list.json'

# 파일 내용 읽기
with open(file_name, 'r') as file:
    json_data = json.load(file)

# JSON 데이터의 개수 확인
data_count = len(json_data)

# 결과 출력
print(f"{file_name} 파일의 데이터 개수: {data_count}")


yongin_list.json 파일의 데이터 개수: 268


In [40]:
# 크롤링데이터 csv파일로 변환
import pandas as pd

# JSON 파일 읽기
with open('yongin_list.json', 'r') as json_file:
    data = json.load(json_file)

# JSON 데이터를 DataFrame으로 변환
df = pd.DataFrame(data)

# CSV 파일로 저장
df.to_csv('용인시고속도로교통사고데이터.csv', index=False)

In [43]:
# 크롤링한 좌표값 변환하는작업
import pandas as pd
from pyproj import Proj, Transformer

# CSV 파일 경로
csv_file_path = '용인시고속도로교통사고데이터.csv'

# CSV 파일 읽기
df = pd.read_csv(csv_file_path)

# epsg:5181 좌표계 정의
in_proj = Proj(init='epsg:5181')

# epsg:4326 좌표계 정의
out_proj = Proj(init='epsg:4326')

# 좌표 변환 함수 정의
def convert_coordinates(row):
    lon, lat = row['lon(경도)'], row['lat(위도)']
    lon, lat = Transformer.from_proj(in_proj, out_proj).transform(lon, lat)
    return pd.Series({'lon(경도)': lon, 'lat(위도)': lat})

# Rename the columns
df.columns = ['lon(경도)', 'lat(위도)']

# 'lon(경도)'와 'lat(위도)' 컬럼 생성 및 좌표 변환
df[['lon(경도)', 'lat(위도)']] = df.apply(convert_coordinates, axis=1)

# 결과를 CSV 파일로 저장
df.to_csv('용인시고속도로교통사고데이터(위도경도).csv', index=False)






c:\Users\sons\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\sons\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [44]:
# 위도경도 순서바꾸기
import pandas as pd

# CSV 파일 경로
csv_file_path = '용인시고속도로교통사고데이터(위도경도).csv'

# CSV 파일 읽기
df = pd.read_csv(csv_file_path)

# 'lon(경도)'와 'lat(위도)' 컬럼의 위치 바꾸기
df = df[['lat(위도)', 'lon(경도)']]

# 결과를 CSV 파일로 저장
df.to_csv('yongin_swapped_result.csv', index=False)


In [46]:
# 사고데이터와 크롤링후 변환한 좌표데이터 붙이기작업

import pandas as pd

# accidentInfoList.csv 파일 읽기
accident_df = pd.read_csv('yongin_accidentInfoList.csv', encoding='cp949')

# swapped_result.csv 파일 읽기(좌표데이터)
coord_df = pd.read_csv('yongin_swapped_result.csv')

# 두 DataFrame을 수평으로 붙이기
merged_df = pd.concat([accident_df, coord_df], axis=1)

# 결과를 CSV 파일로 저장
merged_df.to_csv('yongin_merged_result.csv', index=False)